# Step by Step procedure

Steps 1-8 should be run only once.

1.  In order to not pollute the vector store we will remove the current index:

In [ ]:
from  index_manager import IndexManager
index_manager = IndexManager()
index_manager.delete_index()

2.  In order to use the documentation in the blob storage: 

In [1]:
from storage_manager import AzureBlobStorageManager


manager = AzureBlobStorageManager() # Initialize the manager
# run this line if you want to create a container
# manager.create_container()  # Create a container
# run this line if you want to upload a folder
# manager.upload_directory("name of local folder")  # Upload a directory ex: manager.upload_directory("PDF") 
manager.download_blobs()    # Download all blobs

3.  Process all the documents found in the blob:

In [ ]:
from directory_processor import DirectoryProcessor

directory_processor = DirectoryProcessor(manager.get_download_folder_path())
dataset = directory_processor.process_directory()

4. Initialize LLM models:

In [22]:
from llm_models import AzureOpenAIManager

Model = AzureOpenAIManager()
embeddings = Model.create_embeddings()

5.  Load all the documents:

In [ ]:
from loader_manager import DocumentLoaderManager

loader_manager = DocumentLoaderManager(data=dataset, page_content_column="page_content")

try:
    documents = loader_manager.load_documents_from_dataframe()
except Exception as e:
    print(f"An error occurred: {str(e)}")

6. Split the documents in chunks

In [ ]:
from splitter_manager import DocumentSplitter

splitter = DocumentSplitter()

try:
    # Split the documents
    splits = splitter.split_documents(documents)
except Exception as e:
    print(f"An error occurred: {str(e)}")

7. Vectorize chucnks and added to a vector Store

In [ ]:
from vector_store_manager import VectorStoreManager

vector_store_manager = VectorStoreManager()
vector_test = vector_store_manager.add_documents(splits)


8. Initialize Retriever:

In [ ]:
from retriever_manager import ManagerRetrieve

retriever_manager = ManagerRetrieve()
retriever_manager.create_retriever()
retriever_manager.get_retriever()

9. Ask your question

In [1]:
from QA_manager import QuestionAnswerManager

qa_manager = QuestionAnswerManager()
result = qa_manager.ask("Quelles sont les informations de la cellule 909 du DEG")
print(result["answer"])


La cellule 909 du DEG est située dans l'angle nord-est du hall 706, au niveau -3,06 m. Elle contient les échangeurs A1 et C1 et possède une trappe d'accès depuis la passerelle 706. La trappe est verrouillée selon une consigne d'atelier. La cellule mesure 3,5 mètres de longueur, 2,5 mètres de largeur et 2,5 mètres de hauteur. Des traces de coulures sont présentes à proximité du siphon de sol localisé dans le coin sud-ouest de la cellule.
